In [1]:
import csv
import glob
import json
import os
import re

from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import numpy as np
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
import tqdm

In [2]:
FILE_NAME = 'sentiments.json'
CSV_FOLDER = 'data'
csv.field_size_limit(2147483647)

csv_files = glob.glob(os.path.join(CSV_FOLDER, '*.csv'))
print(csv_files)

['data/NZZS_output.csv', 'data/SRF_output.csv', 'data/SRFV_output.csv', 'data/BZ_output.csv', 'data/BEOLO_output.csv', 'data/BU_output.csv', 'data/SWII_output.csv', 'data/LAL_output.csv', 'data/NAU_output.csv', 'data/TELE_output.csv', 'data/SISP_output.csv', 'data/NNBE_output.csv', 'data/FUW_output.csv', 'data/TAM_output.csv', 'data/BEO_output.csv', 'data/ZSZ_output.csv', 'data/NNBS_output.csv', 'data/LB_output.csv', 'data/NNTA_output.csv', 'data/ZWA_output.csv', 'data/THT_output.csv', 'data/CASO_output.csv', 'data/TAZT_output.csv', 'data/NZZ_output.csv', 'data/SIS_output.csv', 'data/TVS_output.csv', 'data/BEOL_output.csv', 'data/NZZM_output.csv', 'data/APO_output.csv', 'data/NZZF_output.csv', 'data/SF_output.csv', 'data/TVLL_output.csv', 'data/NZZO_output.csv', 'data/SBLI_output.csv', 'data/BLIO_output.csv', 'data/SIG_output.csv', 'data/BIZO_output.csv', 'data/ZHUL_output.csv', 'data/TAS_output.csv', 'data/THTO_output.csv', 'data/BEOO_output.csv', 'data/BOL_output.csv', 'data/SHZO_out

In [3]:
def gen_dict_extract(key, var):
    if hasattr(var, 'items'):
        for k, v in var.items():
            if k == key:
                yield v
            if isinstance(v, dict):
                for result in gen_dict_extract(key, v):
                    yield result
            elif isinstance(v, list):
                for d in v:
                    for result in gen_dict_extract(key, d):
                        yield result

In [4]:
def read_csv( path ):
  try:
    with open(path, encoding='utf-8') as data:
      reader = csv.DictReader(data)
      for row in reader:
        yield row
  except IOError:
    print("ERROR: file [" + path + "] not found/accessible")
    yield ""
    return
  data.close()

In [10]:
model = pickle.load(open('rf.mdl', 'rb'))
vectorizer = pickle.load(open('vocabulary.tfidf', 'rb'))

In [18]:
with open(FILE_NAME) as f:
    sentiments = json.load(f)
# with open('arts_with_sentiments.json', 'r') as f:
#     done_arts = json.load(f)

JSONDecodeError: Expecting value: line 1 column 2 (char 1)

In [12]:
print(sentiments)

[{'file_name': 'APO_output.csv', 'file_idx': 0, 'sentiment': 'positive', 'score': 0.61235}, {'file_name': 'APO_output.csv', 'file_idx': 1, 'sentiment': 'positive', 'score': 0.402463}, {'file_name': 'APO_output.csv', 'file_idx': 2, 'sentiment': 'positive', 'score': 0.955996}, {'file_name': 'APO_output.csv', 'file_idx': 3, 'sentiment': 'negative', 'score': -0.423457}, {'file_name': 'APO_output.csv', 'file_idx': 4, 'sentiment': 'positive', 'score': 0.443104}, {'file_name': 'APO_output.csv', 'file_idx': 5, 'sentiment': 'positive', 'score': 0.693386}, {'file_name': 'APO_output.csv', 'file_idx': 6, 'sentiment': 'neutral', 'score': 0}, {'file_name': 'APO_output.csv', 'file_idx': 7, 'sentiment': 'positive', 'score': 0.567067}, {'file_name': 'APO_output.csv', 'file_idx': 8, 'sentiment': 'positive', 'score': 0.637688}, {'file_name': 'APO_output.csv', 'file_idx': 9, 'sentiment': 'positive', 'score': 0.968044}, {'file_name': 'APO_output.csv', 'file_idx': 10, 'sentiment': 'neutral', 'score': 0}, {'

In [ ]:
for f in csv_files:
    csv_path = f
    if os.path.exists(f'arts_with_sentiments-{os.path.basename(csv_path)}.json'):
        continue
    with open(f'arts_with_sentiments-{os.path.basename(csv_path)}.json', 'w') as save_file:
        save_file.write('[')
    print ( "parsing file [" + csv_path + "]:" )
    for idx, row in tqdm.tqdm(enumerate( read_csv(csv_path) )):
        sentiment = None
        found_sentiments = [x for x in sentiments if x['file_name'] == os.path.basename(csv_path) and x['file_idx'] == idx]
        if len(found_sentiments) > 0:
            sentiment = found_sentiments[0]['sentiment']
            print(f'Sentiment found in train data for {csv_path}-{idx}: {sentiment}')
#         found_arts = [x for x in done_arts if x['short_site'] == str(row['so']) and x['site'] == str(row['so_txt']) and x['title'] == str(row['ht'])]
#         if len(found_arts) > 0:
#             sentiment = found_arts[0]['sentiment']
#             print(f'Sentiment found in already made for {csv_path}-{idx}: {sentiment}')
        try:
            tx = json.loads(row["tx"])
        except Exception as e:
            print("ERROR: failed to parse row #" + str(idx) + " from [" + csv_path + "]")
            break
        lang = str(row["la"])
        if lang == 'en':
            continue
        whole_text = ""
        for tx_elem in tx:
            for elem in gen_dict_extract('text', tx_elem):
                whole_text += elem
        whole_text.replace('\t', '')
        whole_text = ' '.join(whole_text.split())
        if len(whole_text) == 0:
            continue
        if sentiment is None:
            language = 'german'
            tokenizer = ToktokTokenizer()
            text_tokens = tokenizer.tokenize(whole_text)
            pattern = re.compile('[\W_]+')
            text_tokens = [pattern.sub('', x) for x in text_tokens]
            text_tokens = [x for x in text_tokens if x.lower() not in stopwords.words(language) and x.isalnum()]
            stemmer = SnowballStemmer(language)
            text_tokens = [stemmer.stem(x) for x in text_tokens]
            text_to_vectorize = ' '.join(text_tokens)
            text_vector = vectorizer.transform([text_to_vectorize])
            sentiment = model.predict(text_vector)[0]
        short_site = str(row['so'])
        site = str(row['so_txt'])
        title = str(row['ht'])
        pubtime = str(row['pubDateTime'])
        to_save = {
            'site_short': short_site,
            'site': site,
            'publication_time': pubtime,
            'title': title,
            'text': tx,
            'sentiment': sentiment
        }
        with open(f'arts_with_sentiments-{os.path.basename(csv_path)}.json', 'a') as save_file:
                save_file.write(json.dumps(to_save) + ',\n')        
    with open(f'arts_with_sentiments-{os.path.basename(csv_path)}.json', 'a') as save_file:
        save_file.write(']')

0it [00:00, ?it/s]

parsing file [data/NZZS_output.csv]:


3145it [04:51, 10.79it/s]
5it [00:00, 41.78it/s]

parsing file [data/BZ_output.csv]:


13326it [15:08, 14.67it/s]
13it [00:00, 128.22it/s]

...

17251it [30:00,  9.58it/s]
1it [00:00,  8.12it/s]

parsing file [data/SBLI_output.csv]:


3047it [03:51, 13.17it/s]
2it [00:00, 16.93it/s]

parsing file [data/BLIO_output.csv]:


24914it [25:46, 12.73it/s]